In [315]:
### IMPORTS

import os
import glob
import random
import shutil
import sys
from pathlib import Path
import cv2
import numpy as np
import yaml
from matplotlib import pyplot as plt
import matplotlib.patches as patches

In [316]:
### SCRIPT VARIABLES
# get settings

with open('settings.yaml', 'r') as f:
    sets = yaml.safe_load(f)

## IMAGE DATASET PATH
dataset_path = sets['path_main_db'] #'/db/input'

## AMOUNT OF DESIRED IMAGES
im_amount = sets['dataset_size'] # there is 749 images in the database


## DATABASE FOLDER NAME
dataset_destination_folder_name = sets['path_prepreprocessed_db']
desired_size = sets['img_size']

In [317]:
# augument dataset - create more images from single image



def draw_centered_rects(img, centers, crop_w, crop_h):
    fig, ax = plt.subplots()
    ax.imshow(img, cmap='gray')

    for (cy, cx) in centers:
        x = cx - crop_w // 2
        y = cy - crop_h // 2
        rect = patches.Rectangle((y, x), crop_w, crop_h,
                                 linewidth=1.5, edgecolor='red', facecolor='none')
        #ax.plot(cy, cx, 'g*')
        ax.add_patch(rect)

def augument_image(image_path, desired_size, num_img_multiplicator, remove_old=False, debug=False):
    image = cv2.imread(image_path)
    image_name = os.path.split(image_path)[-1]
    image_name = image_name.split('.')[0]


    y_im, x_im = image.shape[:-1]
    image_size = random.randint(desired_size[0], desired_size[1])
    x_shape, y_shape = image_size, image_size
    filter = 1/(image_size**2) * np.ones([image_size, image_size])

    how_many_fit = int((x_im // x_shape) * (y_im // y_shape) * num_img_multiplicator)
    number_images = 1 if how_many_fit == 0 else how_many_fit



    laplaced = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    laplaced = cv2.Laplacian(laplaced, cv2.CV_8U) / 255
    laplaced = cv2.filter2D(laplaced, -1, filter)
    laplaced = (laplaced - laplaced.min()) / (laplaced.max() - laplaced.min())

    mask = np.zeros_like(laplaced, dtype=np.float32)
    mask[y_shape//2:y_im-(y_shape//2), x_shape//2:x_im-(x_shape//2)] = 1
    laplaced = laplaced * mask

    out_indexes = []
    out_images = []
    for i in range(number_images):
        raveled_laplacian = laplaced.ravel()
        norm_raveled_laplacian = raveled_laplacian / np.sum(raveled_laplacian)
        if np.isnan(norm_raveled_laplacian).any():
            print('Warning: NaN found in file {}'.format(image_path))
            continue
        point = np.random.choice((x_im * y_im), 1, p=norm_raveled_laplacian)
        y, x = np.unravel_index(point, (y_im, x_im))
        y_cnt, x_cnt = y[0], x[0]


        min_y = y_cnt - y_shape
        max_y = y_cnt + y_shape
        min_x = x_cnt - x_shape
        max_x = x_cnt + x_shape

        min_y = 0 if min_y < 0 else min_y
        min_x = 0 if min_x < 0 else min_x

        max_y = y_im if max_y > y_im else max_y
        max_x = x_im if max_x > x_im else max_x

        laplaced[min_y:max_y, min_x:max_x] = 0

        new_image = image[y_cnt - (y_shape // 2): y_cnt + (y_shape // 2), x_cnt - (x_shape // 2): x_cnt + (x_shape // 2)]
        out_images.append(new_image)
        out_indexes.append((x_cnt, y_cnt))

        save_name = '{}_{}.bmp'.format(image_name, i)
        save_name = os.path.join(dataset_destination_folder_name, save_name)


        success = cv2.imwrite(save_name, new_image)

    if remove_old:
        os.remove(image_path)
    if debug:
        draw_centered_rects(laplaced, out_indexes, y_shape, x_shape)
        plt.plot(out_indexes, 'g*')

        for img in out_images:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            plt.figure()
            plt.imshow(img)
            plt.show()


In [318]:
from tqdm import tqdm
from multiprocessing import Pool, cpu_count

def augment_image_wrapper(args):
    image_path, desired_size, num_img_multiplicator = args
    augument_image(image_path, desired_size, num_img_multiplicator, remove_old=True, debug=False)

image_multiplicator = .25
images_paths = glob.glob(os.path.join(dataset_destination_folder_name, '*.*'))
image_path = images_paths[1]
desired_size = [256, 512]


args_list = [(p, desired_size, image_multiplicator) for p in images_paths]
#for i, image_path in enumerate(tqdm(images_paths), decs="Processing", unit="img/s"):
    #pass
    #augument_image(image_path, desired_size, num_img_multiplicator=image_multiplicator, remove_old=True, debug=False)
with Pool(processes=cpu_count()) as pool:
    list(tqdm(pool.imap_unordered(augment_image_wrapper, args_list),
              total=len(args_list),
              desc="Augmenting",
              unit="img"))


Augmenting:   6%|▌         | 45/749 [00:47<03:01,  3.88img/s]/tmp/ipykernel_582799/3132165452.py:46: RuntimeWarning: invalid value encountered in divide
  norm_raveled_laplacian = raveled_laplacian / np.sum(raveled_laplacian)


Augmenting:  15%|█▌        | 115/749 [02:25<04:09,  2.54img/s]  /tmp/ipykernel_582799/3132165452.py:46: RuntimeWarning: invalid value encountered in divide
  norm_raveled_laplacian = raveled_laplacian / np.sum(raveled_laplacian)


Augmenting:  29%|██▊       | 215/749 [04:17<10:45,  1.21s/img]

Augmenting:  58%|█████▊    | 438/749 [07:25<03:49,  1.36img/s]/tmp/ipykernel_582799/3132165452.py:46: RuntimeWarning: invalid value encountered in divide
  norm_raveled_laplacian = raveled_laplacian / np.sum(raveled_laplacian)


Augmenting:  68%|██████▊   | 511/749 [08:49<06:00,  1.52s/img]/tmp/ipykernel_582799/3132165452.py:46: RuntimeWarning: invalid value encountered in divide
  norm_raveled_laplacian = raveled_laplacian / np.sum(raveled_laplacian)


Augmenting:  69%|██████▉   | 520/749 [08:54<02:19,  1.64img/s]/tmp/ipykernel_582799/3132165452.py:46: RuntimeWarning: invalid value encountered in divide
  norm_raveled_laplacian = raveled_laplacian / np.sum(raveled_laplacian)


Augmenting: 100%|██████████| 749/749 [16:22<00:00,  1.31s/img]
